In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
data_unida=pd.concat([data,data1]) # Ajusta el nombre del archivo según corresponda
data_unida=data_unida.iloc[:,1:]
data_unida.head()

2024-05-24 11:10:19.551167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_84696/1644232038.py:10: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_84696/1644232038.py:11: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')


,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,tip_rel_1mes,...,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25,id_cluster
0,2016-04-28,ES,H,44,2016-04-29,N,1.0,0,1.0,N,...,0,0,0,0,0,0.0,0.0,0,0,1.0
1,2016-04-28,ES,H,60,2016-04-29,N,1.0,0,1.0,N,...,0,0,0,0,0,0.0,0.0,0,0,1.0
2,2016-04-28,ES,V,53,2016-04-28,N,1.0,0,1.0,N,...,0,0,0,0,0,0.0,0.0,0,0,1.0
3,2016-04-28,ES,H,47,2016-04-28,N,1.0,0,1.0,N,...,0,0,0,0,0,0.0,0.0,0,0,1.0
4,2016-04-28,ES,V,48,2016-04-28,N,1.0,0,1.0,N,...,0,0,0,0,0,0.0,0.0,0,0,1.0
